In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds

In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input

In [ ]:
# food101(음식 이미지) 데이터셋
food101_dataset = tfds.load('food101', split='train', as_supervised=True)

In [ ]:
# Load the ResNet50 model pre-trained on ImageNet dataset
resnet_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

94765736/94765736 [==============================] - 0s 0us/step


In [ ]:
#데이터 전처리 함수 정의
def preprocess_image_with_nutrition(image, label):
    image = tf.image.resize(image, [224, 224])
    image = tf.cast(image, tf.float32) / 255.0

    #label 텐서에서 문자열(string) 텐서로 변환
    label_str = tf.strings.as_string(label)

    # DB를 참고하여 영양성분 정보 매핑
    nutrition_mapping = {
        'bibimbap': {'sugar': 18.3, 'carbs': 98.7, 'protein': 24.6, 'fat': 22},
        'apple_pie': {'sugar': 9, 'carbs': 3, 'protein': 0, 'fat': 0},
        'baby_back_ribs': {'sugar': 14.43, 'carbs': 10.43, 'protein': 84.72, 'fat': 55.61},
        'beef_carpaccio': {'sugar': 7.93, 'carbs': 12.18, 'protein': 14.9, 'fat': 9.64}
        # 모델 연결까지 완결된 뒤에 추가할 예정
    }

    # Flatten the dictionaries into lists of scalar values
    nutrition_list = [list(values.values()) for values in nutrition_mapping.values()]

    # 영양 성분 리스트 텐서로 변환
    nutrition_tensor = tf.constant(nutrition_list, dtype=tf.float32)

    # 영양 정보 레이블에 따라 추출
    default_nutrition = {'sugar': 0, 'carbs': 0, 'protein': 0, 'fat': 0}
    hashable_label_str = tf.strings.to_hash_bucket_fast(label_str, 100)
    nutritional_info = tf.cond(tf.greater(tf.strings.length(label_str), 0),
                               lambda: tf.gather(nutrition_tensor, hashable_label_str),
                               lambda: tf.constant(list(default_nutrition.values()), dtype=tf.float32))

    return image, nutritional_info

In [ ]:
# 데이터셋 매핑
food101_dataset = food101_dataset.map(preprocess_image_with_nutrition)

In [ ]:
# 배치
batch_size = 32
food101_dataset = food101_dataset.shuffle(buffer_size=len(food101_dataset)).batch(batch_size)

In [ ]:
# 모델 정의(Resnet은 X)
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(4)
])

In [ ]:
# Compile the model
model.compile(optimizer='adam',
              loss=tf.keras.losses.MeanSquaredError(),
              metrics=['accuracy'])

In [ ]:
# Train the model
model.fit(food101_dataset, epochs=5)  # Adjust epochs as needed

Epoch 1/5


InvalidArgumentError: Graph execution error:

Detected at node cond/GatherV2_tfg_inlined_cond_0 defined at (most recent call last):
<stack traces unavailable>
indices = 90 is not in [0, 4)
	 [[{{node cond/GatherV2_tfg_inlined_cond_0}}]]
	 [[IteratorGetNext]] [Op:__inference_train_function_5093]